In [ ]:
# --- Dependencias e imports ---
import os
import unicodedata
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display, clear_output, Image

# --- Heurística para detectar automáticamente la carpeta de sigilos ---
# (1) Si existe "./sigilos", se usa.
# (2) Si no, busca recursivamente una carpeta con >= MIN_IMG_COUNT imágenes.
# (3) Si definís SIGILOS_DIR manualmente (string), tiene prioridad.

# >>> OPCIONAL: podés descomentar y setear manualmente:
# SIGILOS_DIR = "ruta/a/mis/sigilos"

MIN_IMG_COUNT = 5
IMG_EXTS = (".png", ".jpg", ".jpeg")

def _count_images_in_dir(d: Path) -> int:
    try:
        return sum(1 for f in d.iterdir() if f.is_file() and f.suffix.lower() in IMG_EXTS)
    except Exception:
        return 0

def _auto_detect_dir(base: Path) -> Path | None:
    # 1) Preferir ./sigilos si existe y tiene imágenes
    sig = base / "sigilos"
    if sig.is_dir() and _count_images_in_dir(sig) >= 1:
        return sig

    # 2) Buscar recursivamente carpetas con >= MIN_IMG_COUNT imágenes
    candidates = []
    for p in base.rglob("*"):
        if p.is_dir():
            n = _count_images_in_dir(p)
            if n >= MIN_IMG_COUNT:
                candidates.append((n, p))
    if candidates:
        # Escoger la que tenga más imágenes
        candidates.sort(reverse=True, key=lambda x: x[0])
        return candidates[0][1]

    return None

# Resolver carpeta de trabajo
BASE_DIR = Path(".").resolve()

# Prioridad: variable SIGILOS_DIR si existe y es válida
try:
    if 'SIGILOS_DIR' in globals() and SIGILOS_DIR:
        SIGILOS_PATH = Path(SIGILOS_DIR).expanduser().resolve()
        if not SIGILOS_PATH.is_dir():
            raise RuntimeError(f"SIGILOS_DIR apunta a algo que no es carpeta: {SIGILOS_PATH}")
    else:
        SIGILOS_PATH = _auto_detect_dir(BASE_DIR)

    if not SIGILOS_PATH:
        raise RuntimeError(
            "No se pudo detectar automáticamente la carpeta de sigilos.\n"
            "Creá una carpeta 'sigilos' junto a este notebook con tus imágenes, "
            "o seteá SIGILOS_DIR = 'ruta/a/mis/sigilos' antes de ejecutar."
        )
except Exception as e:
    # Mensaje claro y no detiene el resto del notebook si querés seguir con otras celdas
    raise RuntimeError(f"Error detectando carpeta de sigilos: {e}")

print(f"Carpeta de sigilos detectada: {SIGILOS_PATH}")

# --- Utilidades de normalización y búsqueda ---
import unicodedata

def _norm(s: str) -> str:
    """
    Normaliza cadenas: casefold, quita acentos y no alfanuméricos.
    """
    return ''.join(
        c for c in unicodedata.normalize('NFKD', s).casefold()
        if c.isalnum()
    )

def buscar_sigilo(nombre: str) -> str | None:
    """
    Busca una imagen cuyo nombre comience con el nombre del sigilo (normalizado)
    dentro de SIGILOS_PATH y subcarpetas. Acepta .png/.jpg/.jpeg (may/min).
    """
    base = _norm(nombre)
    for root, _, files in os.walk(SIGILOS_PATH):
        for f in files:
            if f.lower().endswith(IMG_EXTS):
                if _norm(f).startswith(base):
                    return str(Path(root) / f)
    # Intento directo exacto
    for ext in IMG_EXTS:
        p = SIGILOS_PATH / f"{nombre}{ext}"
        if p.exists():
            return str(p)
    return None

# --- Diccionario con los 50 nombres de Marduk y sus detalles ---
nombres_marduk = {
    "MARDUK": {"descripcion": "Señor de Señores, Amo de los Magos. Solo debe ser invocado en casos de extrema necesidad.", "palabra_poder": "DUGGA"},
    "MARUKKA": {"descripcion": "Conoce todos los secretos del mundo, humanos y divinos.", "palabra_poder": "Invocar con deseo puro."},
    "MARUTUKKU": {"descripcion": "Maestro en las Artes de la Protección, posee la Estrella ARRA.", "palabra_poder": "ARRA"},
    "BARASHAKUSHU": {"descripcion": "Hacedor de Milagros, el más amable y benefactor de los 50.", "palabra_poder": "BAALDURU"},
    "LUGGALDIMERANKIA": {"descripcion": "Puso orden en el caos, comandante de legiones de demonios del viento.", "palabra_poder": "BANUTUKKU"},
    "NARILUGGALDIMMERANKIA": {"descripcion": "Vigilante de los IGIGI y ANUNNAKI, detecta todo lo que sucede en el mundo superior e inferior.", "palabra_poder": "BANRABISHU"},
    "ASARULUDU": {"descripcion": "Portador de la Espada Flamígera, otorga protección en tareas peligrosas.", "palabra_poder": "BANMASKIM"},
    "NAMTILLAKU": {"descripcion": "Posee el conocimiento de resucitar a los muertos y comunicarse con ellos.", "palabra_poder": "BANUTUKUKUTUKKU"},
    "NAMRU": {"descripcion": "Otorga sabiduría y conocimiento en todas las cosas, especialmente en la ciencia de los metales.", "palabra_poder": "BAKAKALAMU"},
    "ASARU": {"descripcion": "Conoce todas las plantas y árboles, y los hace prosperar incluso en desiertos.", "palabra_poder": "BAALPRIKU"},
    "ASARUALIM": {"descripcion": "Brilla como una luz en la oscuridad, otorga consejo y sabiduría en todas las cosas.", "palabra_poder": "BARRMARATU"},
    "ASARUALIMNUNNA": {"descripcion": "Protector en la batalla, otorga armaduras y conocimientos de estrategia militar.", "palabra_poder": "BANATATU"},
    "TUTU": {"descripcion": "Silencia la angustia y da alegría a los corazones rotos.", "palabra_poder": "DIRRIGUGIM"},
    "ZIUKKINNA": {"descripcion": "Da conocimiento sobre los movimientos de las estrellas y su significado.", "palabra_poder": "GIBBILANNU"},
    "ZIKU": {"descripcion": "Otorga riquezas y revela la ubicación de tesoros escondidos.", "palabra_poder": "GIGGIMAGANPA"},
    "AGAKU": {"descripcion": "Puede dar vida al que ya está muerto por un corto periodo de tiempo.", "palabra_poder": "MASHGARZANNA"},
    "TUKU": {"descripcion": "Protector contra maldiciones y ataques mágicos.", "palabra_poder": "MASHSHAMMASHTI"},
    "SHAZU": {"descripcion": "Conoce los pensamientos de los demás y revela secretos ocultos.", "palabra_poder": "MASHSHANANNA"},
    "ZISI": {"descripcion": "Reconciliador de enemigos, silenciador de disputas.", "palabra_poder": "MASHINANNA"},
    "SUHRIM": {"descripcion": "Busca y destruye a los adoradores de los Antiguos.", "palabra_poder": "MASSHANGERGAL"},
    "SUHGURIM": {"descripcion": "Descubre enemigos y puede atacarlos si se le ordena.", "palabra_poder": "MASHSHADAR"},
    "ZAHRIM": {"descripcion": "Guerrero supremo, destructor de ejércitos enteros.", "palabra_poder": "MASHSHAGARANNU"},
    "ZAHGURIM": {"descripcion": "Asesino letal que mata lentamente de formas misteriosas.", "palabra_poder": "MASHTISHADDU"},
    "ENBILULU": {"descripcion": "Conoce los secretos de los ríos y del agua en la tierra.", "palabra_poder": "BANATATU"},
    "EPADUN": {"descripcion": "Señor de la irrigación, trae agua a los lugares secos.", "palabra_poder": "EYUNGINAKANPA"},
    "ENBILULUGUGAL": {"descripcion": "Preside sobre todo crecimiento y abastece a las ciudades hambrientas.", "palabra_poder": "AGGHA"},
    "HEGAL": {"descripcion": "Señor de la agricultura y de la fertilidad humana.", "palabra_poder": "BURDISHU"},
    "SIRSIR": {"descripcion": "Destructor de TIAMAT, maestro sobre la serpiente.", "palabra_poder": "APIRIKUBABADAZUZUKANPA"},
    "MALAH": {"descripcion": "Otorga valentía y coraje en situaciones difíciles.", "palabra_poder": "BACHACHADUGG"},
    "GIL": {"descripcion": "Hace que la cebada crezca y potencia la fertilidad.", "palabra_poder": "AGGABAL"},
    "GILMA": {"descripcion": "Fundador de ciudades, Poseedor del Conocimiento de la Arquitectura, creador de todo lo que es permanente y nunca se mueve.", "palabra_poder": "AKABAL"},
    "AGILMA": {"descripcion": "El que trae la lluvia, puede causar tormentas o lluvias suaves.", "palabra_poder": "MASHSHAYEGURRA"},
    "ZULUM": {"descripcion": "Conoce dónde y cuándo plantar, da consejo en negocios y protege contra fraudes.", "palabra_poder": "ABBABAAL"},
    "MUMMU": {"descripcion": "Otorga conocimientos sobre la condición de la vida antes de la creación y la estructura del universo.", "palabra_poder": "ALALALABAAAL"},
    "ZULUMMAR": {"descripcion": "Otorga gran fuerza y vitalidad.", "palabra_poder": "ANNDARABAAL"},
    "LUGALABDUBUR": {"descripcion": "Destructor de los Dioses de TIAMAT, venció a sus hordas y encadenó a KUTULU.", "palabra_poder": "AGNIBAAL"},
    "PAGALGUENNA": {"descripcion": "Posee infinita inteligencia y conocimiento de lo aún no creado.", "palabra_poder": "ARRABABAAL"},
    "LUGALDURMAH": {"descripcion": "Señor de los Lugares Elevados, vigilante de los cielos y de todo lo que viaja en él.", "palabra_poder": "ARATAAGARBAL"},
    "ARANUNNA": {"descripcion": "Dador de sabiduría, conocedor del Pacto Mágico y de las Leyes.", "palabra_poder": "ARAMANNGI"},
    "DUMUDUKU": {"descripcion": "Poseedor del Bastón de Lapislázuli, conocedor del Nombre y Número Secreto.", "palabra_poder": "ARATAGIGI"},
    "LUGALANNA": {"descripcion": "Conoce la esencia del mundo antes de la división entre Ancianos y Antiguos.", "palabra_poder": "BALDIKHU"},
    "LUGALUGGA": {"descripcion": "Conoce la esencia de todos los espíritus, vivos y muertos.", "palabra_poder": "ZIDUR"},
    "IRKINGU": {"descripcion": "Capturó a KINGU y conoce los orígenes de la humanidad.", "palabra_poder": "BARERIMU"},
    "KINMA": {"descripcion": "Juez de los Dioses, vigilante del cumplimiento del Pacto.", "palabra_poder": "ENGAIGAL"},
    "ESIZKUR": {"descripcion": "Conoce la duración de la vida de todas las cosas.", "palabra_poder": "NENIGEGAL"},
    "GIBIL": {"descripcion": "Señor del fuego y la forja, purificador de los metales y el alma.", "palabra_poder": "BAALAGNITARRA"},
    "ADDU": {"descripcion": "Controlador de tormentas y vientos celestiales.", "palabra_poder": "KAKODAMMU"},
    "ASHARRU": {"descripcion": "Conocedor de los caminos traicioneros y dador de inteligencia sobre el futuro.", "palabra_poder": "BAXTANDABAL"},
    "NEBIRU": {"descripcion": "Guardián de la Puerta de Marduk y regulador de los astros.", "palabra_poder": "DIRGIRGIRI"},
    "NINNUAM": {"descripcion": "Poder supremo de Marduk, juez de jueces.", "palabra_poder": "GASHDIG"}
}

# --- UI robusto con área de salida dedicada ---

import ipywidgets as widgets
from IPython.display import display, clear_output, Image

# Dropdown con claves como lista (evita problemas con dict_keys)
rituales = list(nombres_marduk.keys())
ritual_dropdown = widgets.Dropdown(
    options=rituales,
    value=rituales[0],
    description='Ritual:',
    style={'description_width': 'initial'}
)

ejecutar_button = widgets.Button(description="Ejecutar Ritual")
instrucciones_button = widgets.Button(description="Ver Instrucciones Generales")

# Área de salida dedicada: el UI no desaparece cuando limpiamos
out = widgets.Output()

def _render_instrucciones():
    print("Instrucciones Generales para los Rituales del Necronomicón de Simón:")
    print("1. Escribe el sello del nombre en un papel blanco con tinta negra.")
    print("2. Encuentra un lugar tranquilo, enciende dos velas y un incienso.")
    print("3. Concéntrate en tu objetivo y respira profundamente tres veces.")
    print("4. Pronuncia el llamado sagrado para invocar la entidad.")
    print("5. Mantén la concentración y visualiza tu objetivo cumplido.")
    print("6. Cierra el ritual con las palabras de despedida y agradecimiento.")
    print("7. Guarda el sello en un lugar seguro y evita que otros lo vean.")
    print("8. Registra los resultados en tu diario de rituales.")

def mostrar_instrucciones(_):
    with out:
        clear_output()
        _render_instrucciones()

def realizar_ritual(nombre):
    with out:
        clear_output()
        detalles = nombres_marduk[nombre]
        print(f"\nRitual para {nombre}:")
        print(f"Descripción: {detalles['descripcion']}")
        print("\nPasos a seguir:")
        print("1. Escribe el sello de este nombre en un papel blanco con tinta negra.")
        print("2. Encuentra un lugar tranquilo, enciende dos velas y un incienso.")
        print("3. Concéntrate en tu objetivo y respira profundamente tres veces.")
        print("4. Pronuncia el siguiente llamado:")
        print("   ZI KIA KANPA, ZI ANNA KANPA")
        print(f"   Escúchame, oh {nombre}, ven a mí por los poderes de la palabra {detalles['palabra_poder']}")
        print("   ¡Y contesta mi Pedido Urgente!")
        print("5. Mantén la concentración y visualiza tu objetivo.")
        print("6. Cierra el ritual con:")
        print("   ZI DINGIR KIA KANPA, ZI DINGIR ANNA KANPA")
        print("7. Guarda el sello en un lugar seguro y no permitas que otros lo vean.")
        print("8. Registra el resultado en tu diario de rituales si lo deseas.")

        # Mostrar sigilo (sin romper el UI si falla)
        img_path = buscar_sigilo(nombre)
        if img_path:
            try:
                display(Image(filename=img_path))  # OBLIGATORIO usar filename=
            except Exception as e:
                print(f"[No se pudo mostrar el sigilo: {e}]")
        else:
            print("[No se encontró imagen del sigilo]")

# Bind de callbacks (se re-asignan cada vez que ejecutás esta celda)
ejecutar_button.on_click(lambda b: realizar_ritual(ritual_dropdown.value))
instrucciones_button.on_click(mostrar_instrucciones)

# Controles + salida en un layout estable
controls = widgets.HBox([ritual_dropdown, ejecutar_button, instrucciones_button])
ui = widgets.VBox([controls, out])

display(ui)

# Render inicial opcional
with out:
    clear_output()
    _render_instrucciones()



Carpeta de sigilos detectada: C:\Users\Usuario Contable\Desktop\Codigo\Necronomicon\sigilos
